In [1]:
# if (!requireNamespace("BiocManager", quietly=TRUE)) 
#     install.packages("BiocManager") 

# BiocManager::install("ANCOMBC")

In [2]:
.libPaths(R.home('/opt/homebrew/Caskroom/miniforge/base/lib/R/library'))
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1013. Microbiome Peanut Allergy/P1013.3. Analyses/P1013.3.2. Descriptive Statistics/Output'
cur_date = '011426'

library(readxl)
library(openxlsx)
library(writexl)
library(tidyverse)
library(factoextra)
library(gtsummary)
library(ANCOMBC) 

# reading in files
saliva_df = data.frame(read_excel("Input/Processed_Saliva_Data_090525.xlsx")) %>%
    mutate(Age = as.numeric(Age), IgE = as.numeric(IgE)) 
stool_df = data.frame(read_excel("Input/Processed_Stool_Data_090525.xlsx")) %>%
    mutate(Age = as.numeric(Age), IgE = as.numeric(IgE)) 

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa



In [3]:
head(saliva_df)

,Original_Sample_ID,Subject_No,Sample_ID,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.__.__.__.__.__.__,⋯,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.__.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.s__aphrophilus,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.s__parainfluenzae,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..__,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_347,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_352,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.3..s__bacterium_HMT_351,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.6..s__bacterium_HMT_870
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,FS01baseline,1,Pre_1,Pre,P,F,3.838356,41.00,High,7,⋯,57,13,0,213,775,0,14,246,44,64
2,FS0236mo,2,Post_2,Post,A,DS,1.873973,14.20,Low,63,⋯,698,0,43,98,1657,0,0,423,146,123
3,FS02baseline,2,Pre_2,Pre,A,DS,1.873973,14.20,Low,0,⋯,0,0,46,371,3359,0,0,0,0,112
4,FS0436mo,3,Post_3,Post,P,F,3.715068,67.40,High,34,⋯,0,0,0,0,92,0,200,67,32,201
5,FS04baseline,3,Pre_3,Pre,P,F,3.715068,67.40,High,4,⋯,0,0,0,0,43,414,47,0,2,1050
6,FS0536mo,4,Post_4,Post,A,DS,3.991781,9.59,Low,6,⋯,0,17,0,139,2521,0,0,502,39,101


# Descriptive Statistics

Determining if there are any significant differences in demographic variables based on outcome.

In [4]:
saliva_df$Outcome = as.character(saliva_df$Outcome)
stool_df$Outcome = as.character(stool_df$Outcome)

combined_saliva_df = saliva_df  %>%
    # ended up reassinging 'DS' to 'F' due to small sample size
    mutate(Outcome = ifelse(Outcome == 'DS', 'F', Outcome)) %>%
    # removing those failed the IgE protein
    filter(IgE_Binarized != 'Fail')

combined_stool_df = stool_df  %>%
    # ended up reassinging 'DS' to 'F' due to small sample size
    mutate(Outcome = ifelse(Outcome == 'DS', 'F', Outcome)) %>%
    filter(IgE_Binarized != 'Fail')

# converting to factors
combined_saliva_df$Outcome = factor(combined_saliva_df$Outcome, levels = c('F', 'SU'))
combined_stool_df$Outcome = factor(combined_stool_df$Outcome, levels = c('F', 'SU'))
combined_saliva_df$Treatment = factor(combined_saliva_df$Treatment, levels = c('A', 'P'))
combined_stool_df$Treatment = factor(combined_stool_df$Treatment, levels = c('A', 'P'))
combined_saliva_df$IgE_Binarized = factor(combined_saliva_df$IgE_Binarized, levels = c('Low', 'High'))
combined_stool_df$IgE_Binarized = factor(combined_stool_df$IgE_Binarized, levels = c('Low', 'High'))

head(combined_saliva_df)

,Original_Sample_ID,Subject_No,Sample_ID,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.__.__.__.__.__.__,⋯,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.__.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.s__aphrophilus,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.s__parainfluenzae,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..__,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_347,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_352,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.3..s__bacterium_HMT_351,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.6..s__bacterium_HMT_870
,<chr>,<dbl>,<chr>,<chr>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,FS01baseline,1,Pre_1,Pre,P,F,3.838356,41.00,High,7,⋯,57,13,0,213,775,0,14,246,44,64
2,FS0236mo,2,Post_2,Post,A,F,1.873973,14.20,Low,63,⋯,698,0,43,98,1657,0,0,423,146,123
3,FS02baseline,2,Pre_2,Pre,A,F,1.873973,14.20,Low,0,⋯,0,0,46,371,3359,0,0,0,0,112
4,FS0436mo,3,Post_3,Post,P,F,3.715068,67.40,High,34,⋯,0,0,0,0,92,0,200,67,32,201
5,FS04baseline,3,Pre_3,Pre,P,F,3.715068,67.40,High,4,⋯,0,0,0,0,43,414,47,0,2,1050
6,FS0536mo,4,Post_4,Post,A,F,3.991781,9.59,Low,6,⋯,0,17,0,139,2521,0,0,502,39,101


In [5]:
length(unique(saliva_df$Subject_No))
length(unique(combined_saliva_df$Subject_No))
length(unique(stool_df$Subject_No))
length(unique(combined_stool_df$Subject_No))

[1] 45

[1] 38

[1] 40

[1] 35

In [6]:
combined_saliva_df %>%
    filter(Time_Point == 'Pre') %>%
    tbl_summary(by = Outcome, missing = "no", 
    include = colnames(saliva_df[c(5,7:9)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})",
                               all_categorical() ~ "{n} ({p}%)")) %>%
    add_n() %>% 
    add_p(test = list(all_categorical() ~ "fisher.test")) %>% # adding p value from anova
    as_tibble() #%>%
    #write_xlsx(., "Output/Table_Saliva.xlsx")

The following warnings were returned during `add_p()`:
! For variable `Age` (`Outcome`) and "estimate", "statistic", "p.value",
  "conf.low", and "conf.high" statistics: cannot compute exact p-value with
  ties
! For variable `Age` (`Outcome`) and "estimate", "statistic", "p.value",
  "conf.low", and "conf.high" statistics: cannot compute exact confidence
  intervals with ties
! For variable `IgE` (`Outcome`) and "estimate", "statistic", "p.value",
  "conf.low", and "conf.high" statistics: cannot compute exact p-value with
  ties
! For variable `IgE` (`Outcome`) and "estimate", "statistic", "p.value",
  "conf.low", and "conf.high" statistics: cannot compute exact confidence
  intervals with ties


**Characteristic**,**N**,**F** N = 25,**SU** N = 10,**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Treatment,35,NA,NA,<0.001
A,NA,9 (36%),10 (100%),NA
P,NA,16 (64%),0 (0%),NA
Age,35,2.67 (0.95),2.36 (1.05),0.3
IgE,35,47 (53),14 (35),0.002
IgE_Binarized,35,NA,NA,0.002
Low,NA,11 (44%),10 (100%),NA
High,NA,14 (56%),0 (0%),NA


In [7]:
combined_stool_df %>%
    filter(Time_Point == 'Pre') %>%
    tbl_summary(by = Outcome, missing = "no", 
    include = colnames(saliva_df[c(5,7:9)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})",
                               all_categorical() ~ "{n} ({p}%)")) %>%
    add_n() %>% 
    add_p(test = list(all_categorical() ~ "fisher.test")) %>% # adding p value from anova
    as_tibble() #%>%
    #write_xlsx(., "Output/Table_Stool.xlsx")

The following warnings were returned during `add_p()`:
! For variable `IgE` (`Outcome`) and "estimate", "statistic", "p.value",
  "conf.low", and "conf.high" statistics: cannot compute exact p-value with
  ties
! For variable `IgE` (`Outcome`) and "estimate", "statistic", "p.value",
  "conf.low", and "conf.high" statistics: cannot compute exact confidence
  intervals with ties


**Characteristic**,**N**,**F** N = 23,**SU** N = 10,**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Treatment,33,NA,NA,<0.001
A,NA,6 (26%),10 (100%),NA
P,NA,17 (74%),0 (0%),NA
Age,33,2.50 (0.93),2.15 (0.92),0.3
IgE,33,50 (54),4 (4),<0.001
IgE_Binarized,33,NA,NA,0.001
Low,NA,9 (39%),10 (100%),NA
High,NA,14 (61%),0 (0%),NA


# Descriptive Statistics

I used the [ANCOM-BC2](https://www.bioconductor.org/packages/release/bioc/vignettes/ANCOMBC/inst/doc/ANCOMBC2.html) package to perform a differential abundance analysis ([ref](https://www.nature.com/articles/s41592-023-02092-7#Sec1)) to run a couple of different models. 

## 1. How do baseline microbiome species differ across outcomes? How do these differences vary after controlling for age and IgE protein concentrations?

In [8]:
# making the row names the sample ids
v1_saliva_df = combined_saliva_df %>%
    # only keep participants' baseline samples and those given active treatment
    filter(Time_Point == 'Pre' & Treatment == 'A') %>%
    # making the row names the sample ids
    column_to_rownames(var = 'Sample_ID')

v1_stool_df = combined_stool_df %>%
    filter(Time_Point == 'Pre' & Treatment == 'A') %>%
    column_to_rownames(var = 'Sample_ID')

v1_saliva_df$Outcome = factor(v1_saliva_df$Outcome, levels = c('F', 'SU'))
v1_stool_df$Outcome = factor(v1_stool_df$Outcome, levels = c('F', 'SU'))

head(v1_saliva_df)

,Original_Sample_ID,Subject_No,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.__.__.__.__.__.__,k__Bacteria.p__Absconditabacteria_.SR1..c__Absconditabacteria_.SR1._.C.1..o__Absconditabacteria_.SR1._.O.1..f__Absconditabacteria_.SR1._.F.1..g__Absconditabacteria_.SR1._.G.1..s__bacterium_HMT_874,⋯,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.__.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.s__aphrophilus,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.s__parainfluenzae,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..__,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_347,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_352,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.3..s__bacterium_HMT_351,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.6..s__bacterium_HMT_870
,<chr>,<dbl>,<chr>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pre_2,FS02baseline,2,Pre,A,F,1.873973,14.20,Low,0,16,⋯,0,0,46,371,3359,0,0,0,0,112
Pre_4,FS05baseline,4,Pre,A,F,3.991781,9.59,Low,24,0,⋯,0,0,0,110,1225,40,4,403,35,167
Pre_6,FS07baseline,6,Pre,A,SU,1.419178,1.32,Low,7,0,⋯,0,0,0,2315,1978,0,16,0,5,28
Pre_9,FS10baseline,9,Pre,A,SU,1.389041,0.42,Low,9,0,⋯,0,0,0,1450,4274,0,0,0,0,7
Pre_11,FS12baseline,11,Pre,A,F,1.567123,4.34,Low,8,0,⋯,0,0,0,16,4652,0,0,37,7,31
Pre_14,FS15baseline,14,Pre,A,F,3.912329,105.00,High,2,0,⋯,8,0,0,33,262,0,0,1465,58,56


How many subjects are included in this analysis?

In [9]:
nrow(v1_saliva_df)
nrow(v1_stool_df)

[1] 19

[1] 16

In [10]:
# setting seed for reproducibility
get_ancombc2_results = function(df, formula, value, paired_test, group_var, interest_group){
    # add words!!!
    
    set.seed(123)
    # automatically performs a psuedo log transformation
    value2 = value + 1
    output = ancombc2(data = t(df[,value2:ncol(df)]), 
                      meta_data = df[,1:value],
                      tax_level = 'Species', #taxonomic level of interest
                      fix_formula = formula, # specifying the model
                      pairwise = paired_test, # denotes whether the test is paired
                      group = group_var, # independent variable
                      p_adj_method = "fdr") # Holm-Bonferroni adj p values

    # removing the standard error and test statistics from the results
    removed_cols = c('se_', 'W_', paste0('diff_', group_var))
    final_df = output$res %>%
        select(!starts_with(removed_cols)) %>%
        rename(Name = taxon) %>%
        mutate(OG_Name = Name) %>%
        # cleaning up names
        separate(Name, c("Name", "Name1"), sep = '.g__') %>%
        mutate(across(Name1, ~ str_replace(string = .x, ".s__", " ")),
              across(Name1, ~ str_replace(string = .x, "sp._", "")),
              across(Name1, ~ str_replace(string = .x, ".__", "")),
              Name1 = ifelse(Name == 'k__Bacteria.__.__.__.__.__.__' , 'Unassigned', Name1)) %>%
        # selecting cols of interest
        select(c("Original_Name", "Name1", ends_with(paste0(group_var, interest_group)))) 

    # changing col names
    colnames(final_df)[2:6] = c('Name','logFC', 'P Value', 'P Adj', 'Sensitivity Analysis')
    
    return(final_df)
    }

# calling fn
saliva_raw_model = get_ancombc2_results(v1_saliva_df, 'Outcome', 8, FALSE, 'Outcome', 'SU')
saliva_2cov_model = get_ancombc2_results(v1_saliva_df, 'Age + IgE + Outcome', 8, FALSE, 'Outcome', 'SU')
stool_raw_model = get_ancombc2_results(v1_stool_df, 'Outcome', 9, FALSE, 'Outcome', 'SU')
stool_2cov_model = get_ancombc2_results(v1_stool_df, 'Age + IgE + Outcome', 9, FALSE, 'Outcome', 'SU')

Checking the input data type ...

The input data is of type: matrix

The imported data is in a generic 'matrix'/'data.frame' format.

PASS

Checking the sample metadata ...

The specified variables in the formula: Outcome

The available variables in the sample metadata: Original_Sample_ID, Subject_No, Time_Point, Treatment, Outcome, Age, IgE, IgE_Binarized

PASS

Checking other arguments ...

The number of groups of interest is: 2

Warning message:
“The group variable has < 3 categories 
The multi-group comparisons (global/pairwise/dunnet/trend) will be deactivated”
The sample size per group is: F = 9, SU = 10

PASS

Obtaining initial estimates ...

Estimating sample-specific biases ...

Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: rngtools

ANCOM-BC2 primary results ...

Conducting sensitivity analysis for pseudo-count addition to 0s ...
For taxa that are sign

In [11]:
head(saliva_raw_model)

,OG_Name,Name,logFC,P Value,P Adj,Sensitivity Analysis,diff_robust_OutcomeSU
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>
1,k__Bacteria.__.__.__.__.__.__,Unassigned,0.6095247,0.287383269,0.65929103,TRUE,FALSE
2,k__Bacteria.p__Absconditabacteria_.SR1..c__Absconditabacteria_.SR1._.C.1..o__Absconditabacteria_.SR1._.O.1..f__Absconditabacteria_.SR1._.F.1..g__Absconditabacteria_.SR1._.G.1..s__bacterium_HMT_874,Absconditabacteria_.SR1._.G.1. bacterium_HMT_874,0.3972753,0.345267767,0.70842061,TRUE,FALSE
3,k__Bacteria.p__Absconditabacteria_.SR1..c__Absconditabacteria_.SR1._.C.1..o__Absconditabacteria_.SR1._.O.1..f__Absconditabacteria_.SR1._.F.1..g__Absconditabacteria_.SR1._.G.1..s__bacterium_HMT_875,Absconditabacteria_.SR1._.G.1. bacterium_HMT_875,0.9313236,0.161028844,0.45952134,TRUE,FALSE
4,k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Actinomycetales.f__Actinomycetaceae.g__Actinomyces.__,Actinomyces,-0.3326994,0.571434087,0.92654727,TRUE,FALSE
5,k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Actinomycetales.f__Actinomycetaceae.g__Actinomyces.s__graevenitzii,Actinomyces graevenitzii,-1.8667526,0.002845002,0.08321631,TRUE,FALSE
6,k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Actinomycetales.f__Actinomycetaceae.g__Schaalia.s__lingnae,Schaalia lingnae,-0.9748577,0.013278054,0.19419154,TRUE,FALSE


## Saliva Raw Results

In [12]:
# passed sensitivity filter and is significant 
saliva_raw_model %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
saliva_raw_model %>%
    filter(`P Adj` < 0.1 & `Sensitivity Analysis` == TRUE) %>%
    nrow()

# significant with a relaxed filter
saliva_raw_model %>%
    filter(`P Adj` < 0.1) %>%
    nrow()

[1] 0

[1] 2

[1] 4

## Saliva Adjusted Results

In [13]:
# passed sensitivity filter and is significant 
saliva_2cov_model %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
saliva_2cov_model %>%
    filter(`P Adj` < 0.1 & `Sensitivity Analysis` == TRUE) %>%
    nrow()

# significant with a relaxed filter
saliva_2cov_model %>%
    filter(`P Adj` < 0.1) %>%
    nrow()

[1] 0

[1] 8

[1] 13

## Stool Raw Results

In [14]:
# passed sensitivity filter and is significant 
stool_raw_model %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
stool_raw_model %>%
    filter(`P Adj` < 0.1 & `Sensitivity Analysis` == TRUE) %>%
    nrow()

# significant with a relaxed filter
stool_raw_model %>%
    filter(`P Adj` < 0.1) %>%
    nrow()

[1] 0

[1] 18

[1] 18

## Stool Adjusted Results

In [15]:
# passed sensitivity filter and is significant 
stool_2cov_model %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
stool_2cov_model %>%
    filter(`P Adj` < 0.1 & `Sensitivity Analysis` == TRUE) %>%
    nrow()

# significant with a relaxed filter
stool_2cov_model %>%
    filter(`P Adj` < 0.1) %>%
    nrow()

[1] 0

[1] 8

[1] 13

# 2. What microbiome species associate with IgE concentrations at baseline? How is that affected after controlling for age?

In [16]:
`%notin%` <- Negate(`%in%`)

v2_saliva_df = combined_saliva_df %>%
    # only keep participants' baseline samples 
    filter(Time_Point == 'Pre') %>%
    # making the row names the sample ids
    column_to_rownames(var = 'Sample_ID')

v2_stool_df = combined_stool_df %>%
     filter(Time_Point == 'Pre') %>%
    column_to_rownames(var = 'Sample_ID')

How many subjects do we have?

In [17]:
length(unique(v2_saliva_df$Subject_No))
length(unique(v2_stool_df$Subject_No))

[1] 35

[1] 33

In [18]:
# calling fn
saliva_ige = get_ancombc2_results(v2_saliva_df, 'IgE_Binarized', 8, FALSE, 'IgE_Binarized', 'High')
saliva_ige_cov = get_ancombc2_results(v2_saliva_df, 'Age + IgE_Binarized', 8, FALSE, 'IgE_Binarized', 'High')
stool_ige = get_ancombc2_results(v2_stool_df, 'IgE_Binarized', 9, FALSE, 'IgE_Binarized', 'High')
stool_ige_cov = get_ancombc2_results(v2_stool_df, 'Age + IgE_Binarized', 9, FALSE, 'IgE_Binarized', 'High')

Checking the input data type ...

The input data is of type: matrix

The imported data is in a generic 'matrix'/'data.frame' format.

PASS

Checking the sample metadata ...

The specified variables in the formula: IgE_Binarized

The available variables in the sample metadata: Original_Sample_ID, Subject_No, Time_Point, Treatment, Outcome, Age, IgE, IgE_Binarized

PASS

Checking other arguments ...

The number of groups of interest is: 2

Warning message:
“The group variable has < 3 categories 
The multi-group comparisons (global/pairwise/dunnet/trend) will be deactivated”
The sample size per group is: Low = 21, High = 14

PASS

Obtaining initial estimates ...

Estimating sample-specific biases ...

Warning message in sqrt(sweep(var_hat, 2, var_delta, "*")):
“NaNs produced”
ANCOM-BC2 primary results ...

Conducting sensitivity analysis for pseudo-count addition to 0s ...
For taxa that are significant but do not pass the sensitivity analysis,
they are marked in the 'passed_ss' column and

In [19]:
# passed sensitivity filter and is significant 
saliva_ige %>%
    filter(diff_robust_IgE_BinarizedHigh == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
saliva_ige %>%
    filter(`P Adj` < 0.1 & `Sensitivity Analysis` == TRUE) %>%
    nrow()

# significant with a relaxed filter
saliva_ige %>%
    filter(`P Adj` < 0.1) %>%
    nrow()

[1] 0

[1] 3

[1] 6

In [20]:
# passed sensitivity filter and is significant 
saliva_ige_cov %>%
    filter(diff_robust_IgE_BinarizedHigh == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
saliva_ige_cov %>%
    filter(`P Adj` < 0.1 & `Sensitivity Analysis` == TRUE) %>%
    nrow()

# significant with a relaxed filter
saliva_ige_cov %>%
    filter(`P Adj` < 0.1) %>%
    nrow()

[1] 0

[1] 7

[1] 8

In [21]:
# passed sensitivity filter and is significant 
stool_ige %>%
    filter(diff_robust_IgE_BinarizedHigh == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
stool_ige %>%
    filter(`P Adj` < 0.1 & `Sensitivity Analysis` == TRUE) %>%
    nrow()

# significant with a relaxed filter
stool_ige %>%
    filter(`P Adj` < 0.1) %>%
    nrow()

[1] 0

[1] 15

[1] 50

In [22]:
# passed sensitivity filter and is significant 
stool_ige_cov %>%
    filter(diff_robust_IgE_BinarizedHigh == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
stool_ige_cov %>%
    filter(`P Adj` < 0.1 & `Sensitivity Analysis` == TRUE) %>%
    nrow()

# significant with a relaxed filter
stool_ige_cov %>%
    filter(`P Adj` < 0.1) %>%
    nrow()

[1] 0

[1] 24

[1] 36

# 3. Are there significant changes in the microbiome across outcome groups? Are those changes impacting the outcome using the treatment, age, and IgE as covariates?

In [23]:
# getting dfs first
paired_saliva_df = combined_saliva_df %>%
    # removing 11 participants that don't have pre and post samples
    filter(Subject_No %notin% c(1,16,17,20,24,26,31,32,36,37,43),
    # retaining those given treatment
          Treatment == 'A') %>%
    # making the row names the sample ids
    column_to_rownames(var = 'Sample_ID')

paired_stool_df = combined_stool_df %>%
    # removing 12 participants that don't have pre and post samples
    filter(Subject_No %notin% c(3,10,14,15,23,25,27,32,35,39),
    # retaining those given treatment
          Treatment == 'A') %>%
    # making the row names the sample ids
    column_to_rownames(var = 'Sample_ID')

head(paired_saliva_df)

,Original_Sample_ID,Subject_No,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.__.__.__.__.__.__,k__Bacteria.p__Absconditabacteria_.SR1..c__Absconditabacteria_.SR1._.C.1..o__Absconditabacteria_.SR1._.O.1..f__Absconditabacteria_.SR1._.F.1..g__Absconditabacteria_.SR1._.G.1..s__bacterium_HMT_874,⋯,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.__.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.s__aphrophilus,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.s__parainfluenzae,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..__,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_347,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_352,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.3..s__bacterium_HMT_351,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.6..s__bacterium_HMT_870
,<chr>,<dbl>,<chr>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Post_2,FS0236mo,2,Post,A,F,1.873973,14.20,Low,63,25,⋯,698,0,43,98,1657,0,0,423,146,123
Pre_2,FS02baseline,2,Pre,A,F,1.873973,14.20,Low,0,16,⋯,0,0,46,371,3359,0,0,0,0,112
Post_4,FS0536mo,4,Post,A,F,3.991781,9.59,Low,6,0,⋯,0,17,0,139,2521,0,0,502,39,101
Pre_4,FS05baseline,4,Pre,A,F,3.991781,9.59,Low,24,0,⋯,0,0,0,110,1225,40,4,403,35,167
Post_6,FS0736mo,6,Post,A,SU,1.419178,1.32,Low,2,24,⋯,0,0,0,463,1646,7,0,167,61,437
Pre_6,FS07baseline,6,Pre,A,SU,1.419178,1.32,Low,7,0,⋯,0,0,0,2315,1978,0,16,0,5,28


How many subjects are included in this analysis?

In [24]:
length(unique(paired_saliva_df$Subject_No))
length(unique(paired_stool_df$Subject_No))

[1] 16

[1] 12

In [25]:
# calling fn
paired_saliva_raw = get_ancombc2_results(paired_saliva_df, 'Outcome', 8, TRUE, 'Outcome', 'SU')
paired_saliva_2cov = get_ancombc2_results(paired_saliva_df, 'Age + IgE + Outcome', 8, TRUE, 'Outcome', 'SU')
paired_stool_raw = get_ancombc2_results(paired_stool_df, 'Outcome', 9, TRUE, 'Outcome', 'SU')
paired_stool_2cov = get_ancombc2_results(paired_stool_df, 'Age + IgE + Outcome', 9, TRUE, 'Outcome', 'SU')

Checking the input data type ...

The input data is of type: matrix

The imported data is in a generic 'matrix'/'data.frame' format.

PASS

Checking the sample metadata ...

The specified variables in the formula: Outcome

The available variables in the sample metadata: Original_Sample_ID, Subject_No, Time_Point, Treatment, Outcome, Age, IgE, IgE_Binarized

PASS

Checking other arguments ...

The number of groups of interest is: 2

Warning message:
“The group variable has < 3 categories 
The multi-group comparisons (global/pairwise/dunnet/trend) will be deactivated”
The sample size per group is: F = 14, SU = 18

PASS

Obtaining initial estimates ...

Estimating sample-specific biases ...

Warning message in sqrt(sweep(var_hat, 2, var_delta, "*")):
“NaNs produced”
ANCOM-BC2 primary results ...

Conducting sensitivity analysis for pseudo-count addition to 0s ...
For taxa that are significant but do not pass the sensitivity analysis,
they are marked in the 'passed_ss' column and will be t

## Paired Saliva Raw Results

In [26]:
# passed sensitivity filter and is significant 
paired_saliva_raw %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
paired_saliva_raw %>%
    filter(`P Adj` < 0.1 & `Sensitivity Analysis` == TRUE) %>%
    nrow()

# significant with a relaxed filter
paired_saliva_raw %>%
    filter(`P Adj` < 0.1) %>%
    nrow()

[1] 0

[1] 8

[1] 9

## Paired Adjusted Saliva Results

In [27]:
# passed sensitivity filter and is significant 
paired_saliva_2cov %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
paired_saliva_2cov %>%
    filter(`P Adj` < 0.1 & `Sensitivity Analysis` == TRUE) %>%
    nrow()

# significant with a relaxed filter
paired_saliva_2cov %>%
    filter(`P Adj` < 0.1) %>%
    nrow()

[1] 0

[1] 4

[1] 20

## Paired Stool Results

In [28]:
# passed sensitivity filter and is significant 
paired_stool_raw %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
paired_stool_raw %>%
    filter(`P Adj` < 0.1 & `Sensitivity Analysis` == TRUE) %>%
    nrow()

# significant with a relaxed filter
paired_stool_raw %>%
    filter(`P Adj` < 0.1) %>%
    nrow()

[1] 0

[1] 0

[1] 0

## Paired Stool Adjusted Results

In [29]:
# passed sensitivity filter and is significant 
paired_stool_2cov %>%
    filter(diff_robust_OutcomeSU == TRUE) %>%
    nrow()

# passed sensitivity analysis with a relaxed filter
paired_stool_2cov %>%
    filter(`P Adj` < 0.1 & `Sensitivity Analysis` == TRUE) %>%
    nrow()

# significant with a relaxed filter
paired_stool_2cov %>%
    filter(`P Adj` < 0.1) %>%
    nrow()

[1] 0

[1] 7

[1] 57

In [30]:
#exporting
write.xlsx(saliva_raw_model, paste0(Output,"/", "Saliva_Outcome_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(saliva_2cov_model, paste0(Output,"/", "Saliva_Adj_Outcome_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(stool_raw_model, paste0(Output,"/", "Stool_Outcome_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(stool_2cov_model, paste0(Output,"/", "Stool_Adj_Outcome_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(saliva_ige, paste0(Output,"/", "Saliva_IgE_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(saliva_ige_cov, paste0(Output,"/", "Saliva_Adj_IgE_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(stool_ige, paste0(Output,"/", "Stool_IgE_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(stool_ige_cov, paste0(Output,"/", "Stool_Adj_IgE_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(paired_saliva_raw, paste0(Output,"/", "Saliva_Paired_Outcome_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(paired_saliva_2cov, paste0(Output,"/", "Saliva_Adj_Paired_Outcome_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(paired_stool_raw, paste0(Output,"/", "Stool_Paired_Outcome_", cur_date, ".xlsx"), rowNames = FALSE)
write.xlsx(paired_stool_2cov, paste0(Output,"/", "Stool_Adj_Paired_Outcome_", cur_date, ".xlsx"), rowNames = FALSE)